In [ ]:

import random
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

def trainSVM(X,Y,C,lr):
    timeStamp = 1
    N = np.size(X,0)
    d = np.size(X,1)
    w = np.zeros([d,1])
    b = 0
    lr = float(lr)/timeStamp

    while (timeStamp < 5) and ((prevW - w).T @ (prevW - w)/lr > 1e-5):
        for n in range(N):
            Xn = X[n,0:][None].T
            prevW = w
            # stocastic descent 
            if (1 - Ytr[n]*(w.T @ Xn) + b) > 0:
                w = w - lr*((1./N)*w - C*Y[n]*Xn)
                b = b + lr*(float(C)*Y[n])
            else:
                w = w - lr*(1./N)*w

        timeStamp = timeStamp + 1
        lr = lr/timeStamp
    return w,b
    
def SVMresults(X, Y, w,b):
    Yhat = np.sign(X.dot(w) + b)
    acc = sum(Yhat == Y)/float(np.size(X,0))
    return acc, Y, Yhat

def main():
    train = 22500
    test = 25000
    dev = 2500


    X_train = pd.read_csv('PublicData/reduced_tf_labeledBow.csv')
    y_train = pd.read_csv('PublicData/review_ratings.csv')
    X_test = pd.read_csv('PublicData/test_reduced_tf_labeledBow.csv')
    y_test = pd.read_csv('PublicData/test_review_ratings.csv')

    y_test['Sentiment'] = y_test['Sentiment'].replace(0, -1)
    y_train['Sentiment'] = y_train['Sentiment'].replace(0, -1)

    allIdx = set(range(25000))
    trainingSet = random.sample(allIdx,train)
    devSet = random.sample(allIdx.difference(trainingSet),dev)
    testSet = set(range(25000))


    X_tr = X_train.loc[list(trainingSet),].values
    Y_tr = y_train['Sentiment'].loc[list(trainingSet),].values[None].T
    X_dev = X_train.loc[list(devSet),].values
    Y_dev = y_train['Sentiment'].loc[list(devSet),].values[None].T

    X_te = X_test.loc[list(testSet),].values
    Y_te = y_test['Sentiment'].loc[list(testSet),].values[None].T

    print(X_tr.shape, Y_tr.shape, X_dev.shape, Y_dev.shape, X_te.shape, Y_te.shape)

    i=0
    j=0
    gridsearch = np.zeros([11,8])
    CGrid = np.zeros([11])
    lr_Grid = np.zeros([8])
    
    # Tuning C and Learning rate here
    for C in np.power(2.,range(-7,14,2)):
        for lr in [0.00001, 0.0001, 0.001, 0.01, 0.05, 0.1, 0.5, 1]: # grid search
            print('C =',C, 'lr =',lr)
            w,b = trainSVM(X_tr,Y_tr,C,lr)  # Model building on training set
            gridsearch[i,j], _, _ = SVMresults(X_dev,Y_dev,w,b)
            print('Accuracy', accuGrid[i,j])
            CGrid[i] = C 
            lr_Grid[j] = lr; 
            j=j+1
        j=0
        i=i+1
    i_,j_ = np.where(gridsearch == np.max(gridsearch))

    C_final = CGrid[i_[0]]
    lr_final = lr_Grid[j_[0]]
    
    #final model
    print(C_final, lr_final)
    w_final, b_final = trainSVM(X_tr,Y_tr,C_final,lr_final)

    # Testing the final model in test data
    accuracy, yhat, ytest = SVMresults(X_te,Y_te,w_final,b_final)
    print('Accuracy in test data =',accuracy)
    print(confusion_matrix(yhat, ytest, normalize='true'))
